In [272]:
import datetime
import warnings
warnings.filterwarnings('ignore')

In [273]:
train = pd.read_csv('train_final3.csv')
train2 = pd.read_csv('train_final3.csv')

In [274]:
train.head()

,msno,is_churn,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,num_25,...,gender,registered_via,registration_init_time,age,pro_25,pro_50,pro_75,pro_985,pro_100,disc_user
0,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,36.0,30.0,180.0,0.0,20170311,20170411,0.0,0.000000,...,male,3.0,20131223,20,0.000000,0.125000,0.062500,0.000000,0.812500,0.0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,15.0,90.0,300.0,0.0,20170314,20170615,0.0,239.000000,...,male,3.0,20131227,18,0.430631,0.102703,0.057658,0.039640,0.369369,0.0
2,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,1.0,20151208,20180108,0.0,103.390612,...,female,7.0,20140109,38,0.160336,0.039255,0.024284,0.027049,0.749077,0.0
3,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,41.0,30.0,99.0,1.0,20170316,20170918,0.0,9.000000,...,female,7.0,20140125,35,0.009128,0.007099,0.004057,0.004057,0.975659,0.0
4,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,40.0,30.0,149.0,1.0,20170316,20170314,1.0,6.000000,...,female,3.0,20140126,26,0.122449,0.020408,0.020408,0.081633,0.755102,0.0


In [275]:
train2['transaction_date'] = pd.to_datetime(train2['transaction_date'], format='%Y%m%d')
train2['membership_expire_date'] = pd.to_datetime(train2['membership_expire_date'], format='%Y%m%d')

In [276]:
# num_25 ~ num_100 삭제 (pro25 ~ pro_100으로 대체)
train.drop(labels=['num_25', 'num_50', 'num_75', 'num_985', 'num_100'], axis=1, inplace=True)

In [291]:
# registration_init_time의 이상값 총 108210개
err_value = train2[train2['registration_init_time'] < 10000]
len(err_value)

108210

In [279]:
tmp = train2[train2['registration_init_time'] > 10000]
tmp['registration_init_time'] = pd.to_datetime(tmp['registration_init_time'], format='%Y%m%d')

tran_minus_regit = train2['transaction_date'] - tmp['registration_init_time']

tran_mean_regit = tran_minus_regit.mean() # transaction_date - registration_init_time의 평균값 : 1287days
tran_mean_regit

Timedelta('1287 days 00:16:31.743803')

In [283]:
new_regit_value = err_value['transaction_date'] - tran_mean_regit
new_regit_value = pd.to_datetime(new_regit_value, format='%Y%m%d', box=True)
new_regit_value_ls = []
for i in new_regit_value:
    new_regit_value_ls.append(i.strftime('%Y%m%d'))

In [284]:
new_regit_value_ls[:10]

['20120609',
 '20130826',
 '20130911',
 '20130831',
 '20130909',
 '20130827',
 '20130830',
 '20130901',
 '20130823',
 '20130827']

In [289]:
train.set_value(index=new_regit_value.index, col='registration_init_time', value=new_regit_value_ls)

,msno,is_churn,payment_method_id,payment_plan_days,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,num_unq,...,gender,registered_via,registration_init_time,age,pro_25,pro_50,pro_75,pro_985,pro_100,disc_user
0,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,36.0,30.0,180.0,0.0,20170311,20170411,0.0,30.000000,...,male,3.0,20131223,20,0.000000,0.125000,0.062500,0.000000,0.812500,0.0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,15.0,90.0,300.0,0.0,20170314,20170615,0.0,432.000000,...,male,3.0,20131227,18,0.430631,0.102703,0.057658,0.039640,0.369369,0.0
2,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,1.0,20151208,20180108,0.0,472.744190,...,female,7.0,20140109,38,0.160336,0.039255,0.024284,0.027049,0.749077,0.0
3,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,41.0,30.0,99.0,1.0,20170316,20170918,0.0,548.000000,...,female,7.0,20140125,35,0.009128,0.007099,0.004057,0.004057,0.975659,0.0
4,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,1,40.0,30.0,149.0,1.0,20170316,20170314,1.0,48.000000,...,female,3.0,20140126,26,0.122449,0.020408,0.020408,0.081633,0.755102,0.0
5,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,41.0,30.0,99.0,1.0,20170307,20170307,1.0,472.744190,...,male,7.0,20140129,39,0.160336,0.039255,0.024284,0.027049,0.749077,0.0
6,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,1,38.0,410.0,1788.0,0.0,20170324,20180513,0.0,758.000000,...,female,3.0,20140202,28,0.066838,0.017138,0.025707,0.013710,0.876607,0.0
7,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,1,38.0,90.0,477.0,0.0,20170328,20170707,0.0,915.000000,...,female,9.0,20140212,21,0.260519,0.077887,0.038496,0.034020,0.589078,0.0
8,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,1,40.0,30.0,149.0,1.0,20170326,20170525,0.0,605.000000,...,male,7.0,20140228,20,0.514673,0.038375,0.012415,0.022573,0.411964,0.0
9,t5rqTxCnG7s5VBgEfdkQCezv5KBK7+DMujNibYgylrs=,1,38.0,30.0,149.0,0.0,20170216,20170417,0.0,449.000000,...,male,9.0,20140307,26,0.096436,0.052411,0.048218,0.025157,0.777778,0.0


In [290]:
train['gender'] = train['gender'].replace({'male':0, 'female':1}) # gender: 0, 1로 변환

In [268]:
train.to_csv("train_final4.csv", index=False)